# 5. Monitor a Model Endpoint using **Model Monitor**

* Goals
    * Create a baselining job to learn dataset contraints and statistics
    * Enable data capture to store input data
    * Compare input data to constraints and statistics from the training set
    * Visualize differences in the distributions

In [2]:
%cd /root/sagemaker-workshop-420/notebooks

/root/sagemaker-workshop-420/notebooks


In [13]:
import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.model import Model

In [7]:
boto_session = boto3.Session()
region = boto_session.region_name
sagemaker_session = sagemaker.Session()
role = get_execution_role()
print(role)

arn:aws:iam::209970524256:role/service-role/AmazonSageMaker-ExecutionRole-20200402T065938


In [15]:
BUCKET = 'sagemaker-workshop-420'
PREFIX = 'xgb-churn'

LOCAL_DATA_DIRECTORY = f'../data/{PREFIX}'

print(f"\nArtifacts will be written to/read from s3://{BUCKET}/{PREFIX}")


Artifacts will be written to/read from s3://sagemaker-workshop-420/xgb-churn


## 1. Load the pretrained XGBoost model

In [11]:
xgboost_image_name = get_image_uri(boto_session.region_name, 'xgboost', repo_version='0.90-2')
xgboost_image_name

'257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'

In [12]:
model_s3_path = 's3://sagemaker-workshop-420/xgb-churn/output/xgboost-customer-churn-2020-04-11-12-15-49-863/output/model.tar.gz'

In [14]:
xgboost_model = Model(model_data=model_s3_path,
                      image=xgboost_image_name,
                      role=role,
                      sagemaker_session=sagemaker_session)

---
## 2. Host the Model and Enable Data Capture

Now that we've trained the model, let's deploy it to a hosted endpoint. To monitor the model after it's hosted and serving requests, we'll also add configurations to capture data that is being sent to the endpoint.

In [30]:
import time
from time import strftime, gmtime
from sagemaker.model_monitor import DataCaptureConfig, DatasetFormat, DefaultModelMonitor
from sagemaker.predictor import csv_serializer, RealTimePredictor

In [19]:
data_capture_prefix = f'{PREFIX}/datacapture'

endpoint_name = "xgboost-customer-churn-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(f"EndpointName = {endpoint_name}")

EndpointName = xgboost-customer-churn-2020-04-12-12-20-22


In [21]:
data_capture_config = DataCaptureConfig(enable_capture=True,
                                        sampling_percentage=100,
                                        destination_s3_uri=f's3://{BUCKET}/{BUCKET}')

In [22]:
xgb_predictor = xgboost_model.deploy(initial_instance_count=1, 
                                     instance_type='ml.m4.xlarge',
                                     endpoint_name=endpoint_name,
                                     data_capture_config=data_capture_config)

-------------!

In [28]:
xgboost_model.name

'sagemaker-xgboost-2020-04-12-12-20-55-092'

In [29]:
xgboost_model.endpoint_name

'xgboost-customer-churn-2020-04-12-12-20-22'

---
## 3. Initialize a Predictor 

Initialize a `sagemaker.predictor.RealTimePredictor` so we can make real-time predictions from our model by making an http POST request. We also set up serializers and deserializers for passing our NumPy arrays to the model behind the endpoint.

In [32]:
# Initialize
# 
xgb_predictor = RealTimePredictor(endpoint=xgboost_model.endpoint_name,
                                  sagemaker_session=sagemaker_session,
                                  serializer = csv_serializer,
                                  deserializer=None,
                                  content_type='text/csv')

---
## 4. Invoke the deployed model

Now that we have a hosted endpoint running, we can make real-time predictions from our model by making an http POST request. First let's load in our data. 

Now, we'll loop over our test dataset and collect predictions by invoking the XGBoost endpoint:

In [36]:
print("Sending test traffic to the endpoint {}. \nPlease wait for a minute...".format(endpoint_name))

with open('data/xgb-churn/test_sample.csv', 'r') as f:
    for row in f:
        payload = row.rstrip('\n')
        #print(payload)
        #break
        response = xgb_predictor.predict(data=payload)
        time.sleep(0.5)

Sending test traffic to the endpoint xgboost-customer-churn-2020-04-10-11-04-02. 
Please wait for a minute...


### Verify that data is captured in Amazon S3

When we made some real-time predictions by sending data to our endpoint, we should have also captured that data for monitoring purposes. 

Let's list the data capture files stored in Amazon S3. Expect to see different files from different time periods organized based on the hour in which the invocation occurred. The format of the Amazon S3 path is:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [41]:
import json
from sagemaker.s3 import S3Uploader, S3Downloader

In [39]:
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
print("Found Data Capture Files:")
capture_files = S3Downloader.list("s3://{}/{}".format(BUCKET, current_endpoint_capture_prefix))
print(capture_files)

Found Data Capture Files:
['s3://sagemaker-workshop-420/xgb-churn/datacapture/xgboost-customer-churn-2020-04-10-11-04-02/AllTraffic/2020/04/10/11/13-04-100-72cc8c73-e072-4ddb-aa98-8b73c4003a34.jsonl']


All the data captured is stored in a SageMaker specific json-line formatted file. Next, Let's take a quick peek at the contents of a single line in a pretty formatted json so that we can observe the format a little better.

In [42]:
capture_file = S3Downloader.read_file(capture_files[-1])

print("=====Single Data Capture====")
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2)[:2000])

=====Single Data Capture====
{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "186,0.1,137.8,97,187.7,118,146.4,85,8.7,6,1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,1.1,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50,0.51,0.52,0.53,1.2,1.3,0.54,1.4,0.55",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.014719205908477306",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "5b8d2be2-441b-4264-8e59-5d23a1e52b36",
    "inferenceTime": "2020-04-10T11:13:04Z"
  },
  "eventVersion": "0"
}
